In [1]:
import pandas as pd
import numpy as np

# ===============================
# 1. LOAD DATA
# ===============================
df = pd.read_csv("sample_market_price_data.csv")

print("Initial Shape:", df.shape)

# ===============================
# 2. HANDLE MISSING VALUES
# ===============================

# ---- Check missing values
print("\nMissing Values Before Cleaning:")
print(df.isnull().sum())

# ---- Derive missing Currency_Code based on Exchange_Code
df['Currency_Code'] = np.where(
    (df['Currency_Code'].isnull()) & (df['Exchange_Code'] == 'NASDAQ'),
    'USD',
    df['Currency_Code']
)

# ---- Fill missing Price using MEDIAN of same Security_ID
df['Price'] = df.groupby('Security_ID')['Price'].transform(
    lambda x: x.fillna(x.median())
)

# ===============================
# 3. CHECK & REMOVE DUPLICATES
# ===============================
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]

print(f"\nDuplicates Removed: {before - after}")

# ===============================
# 4. STANDARDIZE FORMATS
# ===============================

# ---- Standardize Date format
df['Price_Date'] = pd.to_datetime(df['Price_Date'], errors='coerce')

# ---- Standardize Security_ID
df['Security_ID'] = df['Security_ID'].str.upper()

# ---- Ensure numeric Price
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# ===============================
# 5. CURRENCY CONVERSION
# ===============================

# ---- Currency Conversion Table (Sample)
currency_df = pd.DataFrame({
    'Currency_Code': ['USD', 'EUR', 'INR'],
    'Conversion_Rate': [1.0, 1.08, 0.012]
})

# ---- Merge conversion rate
df = df.merge(currency_df, on='Currency_Code', how='left')

# ---- Convert price to USD
df['Price_USD'] = df['Price'] * df['Conversion_Rate']

# ===============================
# 6. FINAL VALIDATION
# ===============================
print("\nMissing Values After Cleaning:")
print(df.isnull().sum())

print("\nFinal Shape:", df.shape)
print("\nCleaned Data Sample:")
print(df.head())

# ===============================
# 7. SAVE CLEAN DATA
# ===============================
df.to_csv("cleaned_market_price_data.csv", index=False)
print("\nCleaned data saved as cleaned_market_price_data.csv")


Initial Shape: (300, 9)

Missing Values Before Cleaning:
Security_ID       0
Vendor_ID         0
Vendor_Code       0
Source_Feed_ID    0
Price_Type        0
Exchange_Code     0
Price_Date        0
Currency_Code     0
Price             0
dtype: int64

Duplicates Removed: 0

Missing Values After Cleaning:
Security_ID        0
Vendor_ID          0
Vendor_Code        0
Source_Feed_ID     0
Price_Type         0
Exchange_Code      0
Price_Date         0
Currency_Code      0
Price              0
Conversion_Rate    0
Price_USD          0
dtype: int64

Final Shape: (300, 11)

Cleaned Data Sample:
  Security_ID Vendor_ID Vendor_Code Source_Feed_ID Price_Type Exchange_Code  \
0        AAPL        V1       YAHOO        FEED_V1      CLOSE        NASDAQ   
1        AAPL        V2       ALPHA        FEED_V2      CLOSE        NASDAQ   
2        AAPL        V3     FINDHUB        FEED_V3      CLOSE        NASDAQ   
3        AAPL        V4       BLOOM        FEED_V4      CLOSE        NASDAQ   
4        A

In [2]:
import pandas as pd
import numpy as np

# ===============================
# 1. LOAD CLEANED DATA
# ===============================
df = pd.read_csv("cleaned_market_price_data.csv")

# ===============================
# 2. AGGREGATION: MAX, MIN, MEAN, MEDIAN, STD
# ===============================
agg_df = df.groupby(
    ['Security_ID', 'Source_Feed_ID', 'Vendor_ID',
     'Price_Type', 'Exchange_Code', 'Price_Date']
).agg(
    Max_Price=('Price_USD', 'max'),
    Min_Price=('Price_USD', 'min'),
    Mean_Price=('Price_USD', 'mean'),
    Median_Price=('Price_USD', 'median'),
    Std_Dev=('Price_USD', 'std')
).reset_index()

# ===============================
# 3. MERGE BACK TO ORIGINAL DATA
# ===============================
df = df.merge(
    agg_df,
    on=['Security_ID', 'Source_Feed_ID', 'Vendor_ID',
        'Price_Type', 'Exchange_Code', 'Price_Date'],
    how='left'
)

# ===============================
# 4. PRICE VARIATION (%)
# ===============================
df['Price_Variation_Pct'] = (
    (df['Price_USD'] - df['Mean_Price']) / df['Mean_Price']
) * 100

df['Price_Variation_Pct'] = df['Price_Variation_Pct'].abs()

# ===============================
# 5. OUTLIER BUCKETING
# ===============================
def outlier_bucket(variation):
    if variation < 3:
        return "< 3%"
    elif 3 <= variation <= 5:
        return "3% - 5%"
    else:
        return "> 5%"

df['Outlier_Bucket'] = df['Price_Variation_Pct'].apply(outlier_bucket)

# ===============================
# 6. CONCURRENT PRICE CALCULATION
# ===============================
concurrent_df = df.groupby(
    ['Security_ID', 'Price_Date']
).agg(
    Max_Price_All=('Price_USD', 'max'),
    Min_Price_All=('Price_USD', 'min'),
    Median_Price_All=('Price_USD', 'median')
).reset_index()

concurrent_df['Concurrent_Price_Flag'] = np.where(
    (concurrent_df['Max_Price_All'] ==
     concurrent_df['Min_Price_All']) &
    (concurrent_df['Min_Price_All'] ==
     concurrent_df['Median_Price_All']),
    1,
    0
)

# ===============================
# 7. LOAD INTO REPORTING TABLES
# ===============================

# Reporting Table 1: Vendor Price KPIs
df.to_csv("reporting_vendor_price_kpi.csv", index=False)

# Reporting Table 2: Concurrent Price Summary
concurrent_df.to_csv("reporting_concurrent_price.csv", index=False)

print("STEP 2 Completed Successfully")


STEP 2 Completed Successfully


In [3]:
import pandas as pd

# ==========================================
# 1. LOAD KPI DATA (FROM STEP 2 MAIN OUTPUT)
# ==========================================
df = pd.read_csv("reporting_vendor_price_kpi.csv")

print("Input KPI Data Shape:", df.shape)

# ==========================================
# 2. AGGREGATE OUTLIER COUNTS
# ==========================================
outlier_agg = df.groupby(
    [
        'Vendor_ID',
        'Vendor_Code',
        'Price_Date',
        'Exchange_Code',
        'Price_Type',
        'Outlier_Bucket'
    ]
).agg(
    Outlier_Security_Count=('Security_ID', 'nunique')
).reset_index()

print("Outlier Aggregation Completed")

# ==========================================
# 3. PIVOT FOR REPORTING
# ==========================================
outlier_pivot = outlier_agg.pivot_table(
    index=[
        'Vendor_ID',
        'Vendor_Code',
        'Price_Date',
        'Exchange_Code',
        'Price_Type'
    ],
    columns='Outlier_Bucket',
    values='Outlier_Security_Count',
    fill_value=0
).reset_index()

# Remove column index name
outlier_pivot.columns.name = None

# ==========================================
# 4. RENAME OUTLIER BUCKET COLUMNS
# ==========================================
outlier_pivot.rename(columns={
    '< 3%': 'Outlier_Less_Than_3_Pct',
    '3% - 5%': 'Outlier_3_To_5_Pct',
    '> 5%': 'Outlier_Greater_Than_5_Pct'
}, inplace=True)

# ==========================================
# 5. ENSURE ALL OUTLIER COLUMNS EXIST (KEY FIX)
# ==========================================
required_columns = [
    'Outlier_Less_Than_3_Pct',
    'Outlier_3_To_5_Pct',
    'Outlier_Greater_Than_5_Pct'
]

for col in required_columns:
    if col not in outlier_pivot.columns:
        outlier_pivot[col] = 0

# ==========================================
# 6. CALCULATE TOTAL OUTLIERS
# ==========================================
outlier_pivot['Total_Outliers'] = (
    outlier_pivot['Outlier_Less_Than_3_Pct'] +
    outlier_pivot['Outlier_3_To_5_Pct'] +
    outlier_pivot['Outlier_Greater_Than_5_Pct']
)

# ==========================================
# 7. DERIVE VENDOR PERFORMANCE SCORE
# ==========================================
def vendor_performance(total_outliers):
    if total_outliers < 5:
        return 'Good'
    elif total_outliers <= 10:
        return 'Average'
    else:
        return 'Poor'

outlier_pivot['Vendor_Performance'] = outlier_pivot['Total_Outliers'].apply(vendor_performance)

# ==========================================
# 8. SORT FOR REPORTING & TREND ANALYSIS
# ==========================================
outlier_pivot.sort_values(
    by=['Price_Date', 'Vendor_ID', 'Price_Type'],
    inplace=True
)

# ==========================================
# 9. LOAD INTO REPORTING TABLE
# ==========================================
outlier_pivot.to_csv(
    "reporting_vendor_outlier_summary.csv",
    index=False
)

print("STEP 2 (Continued) completed successfully")
print("Final Reporting Table Shape:", outlier_pivot.shape)


Input KPI Data Shape: (300, 18)
Outlier Aggregation Completed
STEP 2 (Continued) completed successfully
Final Reporting Table Shape: (150, 10)


In [4]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

In [5]:
csv_path = "reporting_vendor_outlier_summary.csv"
df = pd.read_csv(csv_path)

print("CSV loaded successfully")
print(df.head())

CSV loaded successfully
  Vendor_ID Vendor_Code  Price_Date Exchange_Code Price_Type  \
0        V1       YAHOO  2025-01-01        NASDAQ      CLOSE   
1        V2       ALPHA  2025-01-01        NASDAQ      CLOSE   
2        V3     FINDHUB  2025-01-01        NASDAQ      CLOSE   
3        V4       BLOOM  2025-01-01        NASDAQ      CLOSE   
4        V5     REFINIT  2025-01-01        NASDAQ      CLOSE   

   Outlier_Less_Than_3_Pct  Outlier_3_To_5_Pct  Outlier_Greater_Than_5_Pct  \
0                      2.0                   0                           0   
1                      2.0                   0                           0   
2                      2.0                   0                           0   
3                      2.0                   0                           0   
4                      2.0                   0                           0   

   Total_Outliers Vendor_Performance  
0             2.0               Good  
1             2.0               Good  
2    

# New Section